In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import widgets,interact, interactive, interact_manual, interactive_output
from IPython.display import display, HTML
import datetime as dt
import sqlite3

conn = sqlite3.connect('data.db')
alldata = pd.read_sql('select * from TEJ_financial_statement_seasonal', conn)
#只取出我們需要的欄位
alldata['TSE新產業_名稱']=alldata['TSE新產業_名稱'].str.strip()
alldata['TSE新產業_代碼']=alldata['TSE新產業_代碼'].str.strip()
alldata['市場別']=alldata['市場別'].str.strip()
alldata=alldata[alldata['TSE新產業_名稱'].notna()]
alldata=alldata[alldata['TSE新產業_代碼'].notna()]
alldata['產業名代碼']=alldata['TSE新產業_代碼']+' '+alldata['TSE新產業_名稱']
alldata['產業名代碼']=alldata['產業名代碼'].str.strip()
#alldata['產業名代碼'].replace('      ', np.nan, inplace=True)
alldata=alldata[alldata['產業名代碼'].notna()]

alldata['公司名代碼']=alldata['permno']+' '+alldata['sname']


alldata['lpermno']=alldata['permno'].apply(len)
#只保留股票代碼為4碼，且代碼範圍介於0050~9999之間。
alldata=alldata[(alldata['permno'] >= '0050')&(alldata['permno'] <= '9999')& (alldata['lpermno'] == 4)]
alldata['date'] = pd.to_datetime(alldata['ym'].astype(str), format='%Y%m')
alldata['year'] = alldata['date'].dt.year
alldata['權益乘數']=(1/(1-(alldata['負債比率']/100)))*100
#alldata['銷貨利潤邊際']=alldata['ROE－綜合損益']/(alldata['總資產週轉次數']*alldata['權益乘數'])
alldata['季底普通股市價']=alldata['季底普通股市值']/alldata['加權平均股數']
alldata['年-季']=(alldata['year'].astype(str))+'-'+(alldata['season'].astype(str))
#ALL = '全部'
def unique_sorted_values_plus_ALL(array):
    #把pandas其中一欄非重複的資料找出來並且轉成numpy的array，接著再從array轉成list格式
    unique = array.unique().tolist()
    #由小到大去排序
    unique.sort()
    #在索引值0的位置插入'ALL'這文字
    #unique.insert(0, ALL)
    return unique

style = {'description_width': 'initial'}
start = widgets.DatePicker(
    #value = stemp.result.index.min().date(),
    value = dt.datetime(2016, 1, 1),
    description='開始日：',
    disabled=False,style=style
)
end = widgets.DatePicker(
    #value = stemp.result.index.max().date(),
    value=dt.datetime(2021, 3, 25),
    description='結束日：',
    disabled=False,style=style
)

ind_firm=alldata[['產業名代碼','公司名代碼']].drop_duplicates(['產業名代碼', '公司名代碼'], keep='last')
dropdown_industry = widgets.Dropdown(description='請選擇產業代碼：',options =    unique_sorted_values_plus_ALL(alldata['產業名代碼']), value='M1100 水泥工業')
dropdown_company = widgets.Dropdown(description='請選擇公司代碼：',options =    unique_sorted_values_plus_ALL(alldata['公司名代碼']), value='1101 台泥')



# 當使用者選擇某個產業後，公司的下拉選單自動更新成只有該產業的所有公司
def update_dropdown(change):
    dropdown_company.options = tuple(ind_firm[ind_firm['產業名代碼']==dropdown_industry.value]['公司名代碼'])
dropdown_industry.observe(update_dropdown, names='value')

#column_names=list(alldata.columns)
balance_sheet=['年-季','sname','  現金及約當現金', '  透過損益按公允價值衡量之金融資產－流動', '  合約資產－流動', '  應收帳款及票據', '  其他應收款', '  資金貸予他人－流動', '  存貨', '  待出售非流動資產', '  當期所得稅資產－流動', '流動資產', '  按攤銷後成本衡量之金融資產－非流動', '  採權益法之長期股權投資', '  預付投資款', '  不動產廠房及設備', '  商譽及無形資產合計', '    遞延所得稅資產', '    投資性不動產淨額', '  其他非流動資產', '非流動資產', '資產總額', '  短期借款', '  透過損益按公允價值衡量之金融負債－流動', '  合約負債－流動', '  應付帳款及票據', '  其他應付款', '  當期所得稅負債', '  與待出售非流動資產直接相關之負債', '  一年內到期長期負債', '流動負債', '  透過損益按公允價值衡量之金融負債－非流動', '  應付公司債－非流動', '  銀行借款－非流動', '  負債準備－非流動', '  應計退休金負債', '  遞延所得稅', '非流動負債', '負債總額', '  股本', '  資本公積合計', '    法定盈餘公積', '    特別盈餘公積', '    未分配盈餘', '  保留盈餘', '  其他權益', '  庫藏股票帳面值', '母公司股東權益合計', '共同控制下前手權益', '合併前非屬共同控制股權', '非控制權益', '股東權益總額', '負債及股東權益總額']
income_stat=['年-季','sname','營業收入淨額', '營業成本', '營業毛利', '營業費用', '  研究發展費', '  預期信用減損（損失）利益－營業費用', '營業利益', '其他收入', '其他利益及損失', '財務成本', '採權益法之關聯企業及合資損益之份額', '預期信用減損（損失）利益', '除列按攤銷後成本衡量金融資產淨損益', '金融資產重分類淨損益', '營業外收入及支出－其他', '營業外收入及支出', '稅前淨利', '所得稅費用', '繼續營業單位損益', '停業單位損益', '合併前非屬共同控制股權損益', '其他損益調整項－非常項目及累計影響數', '合併總損益', '其他綜合損益－OCI', '本期綜合損益總額', '歸屬母公司淨利（損）', '歸屬非控制權益淨利（損）', '歸屬共同控制下前手權益淨利（損）', '綜合損益歸屬母公司', '綜合損益歸屬非控制權益', '綜合損益歸屬共同控制下前手權益', '每股盈餘', '加權平均股數', '發放特別股股息', '稀釋稅後淨利', '每股盈餘－完全稀釋', '加權平均股數－稀釋', '稅前息前淨利', '稅前息前折舊前淨利', '常續性稅後淨利']
#1. 流動性比率
liq=['年-季','sname','流動比率', '速動比率']
#2.資產管理比率
asset=['年-季','sname','存貨週轉率（次）', '平均售貨天數','應收帳款週轉次數', '平均收帳天數','固定資產週轉次數','總資產週轉次數']
#3. 負債管理比率，沒有固定支出涵蓋比率
liab=['年-季','sname','權益乘數','負債比率','利息保障倍數','利息支出率']  
#4. 獲利能力比率
profit=['年-季','sname','ROA(C)稅前息前折舊前', 'ROA(A)稅後息前', 'ROA(B)稅後息前折舊前', 'ROA－綜合損益', 'ROE(A)－稅後', '稅前息前折舊前淨利率', 'ROE(B)－常續利益', 'ROE－綜合損益', '營業毛利率', '已實現銷貨毛利率', '營業利益率', '稅前淨利率', '稅後淨利率', '業外收支/營收', '常續利益率－稅後', '貝里比率', '營業資產報酬率']
#5. 市場價值比率
market=['年-季','sname','每股淨值(F)－TSE公告數','當季季底P/E', '當季季底P/B', '當季季底PSR', '股利殖利率', '現金股利率', '每股營業額', '每股營業利益', '每股稅前淨利', '每股綜合損益', '季底每股稅前淨利', '季底每股稅後淨利']

## 要比較的財務比率
ratios=['ROE(A)－稅後', 'ROE(B)－常續利益','ROE－綜合損益','稅後淨利率','總資產週轉次數','權益乘數','ROA－綜合損益','每股盈餘','當季季底P/E', '當季季底P/B', '當季季底PSR']
iratios=alldata.groupby(['產業名代碼','date'])[ratios].mean().reset_index()

#稅後淨利率

dropdown_ratios=widgets.Dropdown(
    options=ratios,
    value='ROE－綜合損益',
    description='請選擇財務比率:',style=style
)


output1 = widgets.Output()
output2 = widgets.Output()
ratio1 = widgets.Output()
ratio2 = widgets.Output()
ratio3 = widgets.Output()
ratio4 = widgets.Output()
ratio5 = widgets.Output()

ratioplot = widgets.Output()
plt.style.use('default')

def f(industry,company,start, end, rationame):
    output1.clear_output()
    output2.clear_output()
    ratio1.clear_output()
    ratio2.clear_output()
    ratio3.clear_output()
    ratio4.clear_output()
    ratio5.clear_output()
    ratioplot.clear_output()
    
    filterdata=alldata[(alldata['產業名代碼'] == industry) & (alldata['公司名代碼'] == company) & (alldata['date'] >= pd.to_datetime(start)) & (alldata['date'] <= pd.to_datetime(end))].copy()
    ifilterdata=iratios[(iratios['產業名代碼'] == industry) & (iratios['date'] >= pd.to_datetime(start)) & (iratios['date'] <= pd.to_datetime(end))][['date',rationame]].rename({rationame:'industry_ratio'}, axis=1)
    combine= pd.merge(filterdata[['年-季','date',rationame,'季底普通股市價']], \
                      ifilterdata[['date','industry_ratio']], how='left', \
                      left_on=['date'], right_on=['date'])
    
    tdata1=filterdata[balance_sheet].set_index(['年-季']).T.iloc[:,-8:]  #最多顯示兩年
    tdata2=filterdata[income_stat].set_index(['年-季']).T.iloc[:,-8:]    #最多顯示兩年
    rdata1=filterdata[liq].set_index(['年-季']).T
    rdata2=filterdata[asset].set_index(['年-季']).T
    rdata3=filterdata[liab].set_index(['年-季']).T
    rdata4=filterdata[profit].set_index(['年-季']).T
    rdata5=filterdata[market].set_index(['年-季']).T
    with output1:
        display(tdata1)
    with output2:
        display(tdata2)
    with ratio1:
        display(rdata1)
    with ratio2:
        display(rdata2)
    with ratio3:
        display(rdata3)
    with ratio4:
        display(rdata4)
    with ratio5:
        display(rdata5)
    with ratioplot:
        plt.rcParams["figure.figsize"] = (10,5)
        fig, ax1 = plt.subplots()
        color = 'tab:red'
        ax1.set_xlabel('Season')
        ax1.set_ylabel('Financial Ratio', color=color)
        ax1.plot(combine['年-季'], combine[rationame], color=color, alpha=.8, label='Financial Ratio')
        
        ax1.plot(combine['年-季'], combine['industry_ratio'], color='tab:green', alpha=.8, label='Industry Financial Ratio')
        #ax1.bar(combine['年-季'], combine[industry_ratio], width=0.2, color=color,  alpha=.8)
        ax1.tick_params(axis='y', labelcolor=color)
        ax1.legend()
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color = 'tab:blue'
        ax2.set_ylabel('Stock Price', color=color)  # we already handled the x-label with ax1
        ax2.bar(combine['年-季'], combine['季底普通股市價'], width=0.2, facecolor = 'lightskyblue', edgecolor = 'white', alpha=.4)
        ax2.tick_params(axis='y', labelcolor=color)
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()
        display(combine)

out = interactive(f, industry=dropdown_industry, company=dropdown_company, start=start, end=end, rationame=dropdown_ratios)
#out = interactive_output(f, {'industry': dropdown_industry, 'company': dropdown_company, 'start': start, 'end':end})
#display(dropdown_industry,dropdown_company,start,end,output1, output2, ratio1, ratio2, ratio3, ratio4, ratio5)

item_layout = widgets.Layout(margin='0 0 50px 0')
tab = widgets.Tab([output1, output2, ratio1, ratio2, ratio3, ratio4, ratio5, widgets.VBox([dropdown_ratios,ratioplot])],layout=item_layout)
tab.set_title(0, '資產負債表')
tab.set_title(1, '綜合損益表')
tab.set_title(2, '流動性比率')
tab.set_title(3, '資產管理比率')
tab.set_title(4, '負債管理比率')
tab.set_title(5, '獲利能力比率')
tab.set_title(6, '市場價值比率')
tab.set_title(7, '財務比率趨勢分析')

dashboard = widgets.VBox([widgets.HBox([widgets.VBox([dropdown_industry,dropdown_company]),widgets.VBox([start,end])]),tab])
display(dashboard)